In [144]:
from typing import *
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal
from scipy.stats import norm

from functools import partial

tfd = tfp.distributions

base_dir = "/".join(os.getcwd().split("/")[:-1])

In [145]:
def update_add(tensor: tf.Tensor, index: List[int], update: tf.Tensor):
    
    index = tf.convert_to_tensor([index])
    update = tf.convert_to_tensor([update])
    
    index = tf.convert_to_tensor(index)
    updated = tf.tensor_scatter_nd_add(
        tensor=tensor,
        indices=index,
        updates=update,
    )
    
    return updated


def vector_dot(a: tf.Tensor, b: tf.Tensor):
    return tf.reduce_sum(a*b)


@tf.function(jit_compile=True)
def single_sample_cdf(
        mean: tf.Tensor,
        cov_chol: tf.Tensor,
        lower: tf.Tensor,
        upper: tf.Tensor,
        samples: tf.Tensor,
        verbose: bool = False,
    ):
    
    # Identify data type to use for all calculations
    dtype = mean.dtype
    
    # Dimension of the integral
    D = mean.shape[-1]
    
    # Rename samples, limits and covariance cholesky for brevity
    w = samples
    a = lower
    b = upper
    C = cov_chol
    
    # Initialise transformation variables
    d = tf.zeros_like(mean)
    e = tf.zeros_like(mean)
    f = tf.zeros_like(mean)
    y = tf.zeros_like(mean)
    
    # Initialise standard normal for computing CDFs
    normal = tfp.distributions.Normal(
        loc=tf.zeros(shape=(), dtype=dtype),
        scale=tf.ones(shape=(), dtype=dtype),
    )
    Phi = lambda x: normal.cdf(x)
    iPhi = lambda x: normal.quantile(x)
    
    # Compute transformation variables at the first step
    d = update_add(d, [0], Phi(a[0] / C[0, 0]))
    e = update_add(e, [0], Phi(b[0] / C[0, 0]))
    f = update_add(f, [0], e[0] - d[0])

    for i in tf.range(1, D):
        
        # Update y[i-1]
        y = update_add(y, [i-1], iPhi(d[i-1] + w[i-1] * (e[i-1] - d[i-1])))
        
        # Update d[i-1] and e[i-1]
        d = update_add(d, [i], Phi((a[i] - vector_dot(C[i, :i], y[:i])) / C[i, i]))
        e = update_add(e, [i], Phi((b[i] - vector_dot(C[i, :i], y[:i])) / C[i, i]))
        f = update_add(f, [i], (e[i] - d[i]) * f[i-1])
        
    return f[-1]


def mc_mvn_cdf(
        mean: tf.Tensor,
        cov: tf.Tensor,
        lower: tf.Tensor,
        upper: tf.Tensor,
        num_samples: int,
    ):
    
    samples = tfd.MultivariateNormalFullCovariance(
        loc=mean,
        covariance_matrix=cov,
    ).sample(sample_shape=[num_samples])
    
    lt = tf.reduce_all(tf.math.less(lower, samples), axis=1)
    gt = tf.reduce_all(tf.math.less(samples, upper), axis=1)
    
    return tf.cast(tf.math.logical_and(lt, gt), dtype=tf.float32)


@tf.function
def mvn_cdf(
        mean: tf.Tensor,
        cov: tf.Tensor,
        upper: tf.Tensor,
        num_sobol: int = 100,
        verbose=False,
    ):
    
    B, D = mean.shape
    dtype = mean.dtype
    
    chol = tf.linalg.cholesky(cov)
    lower = float("-inf") * tf.ones_like(upper)
    
    samples = tf.math.sobol_sample(
        dim=D,
        num_results=num_sobol,
        dtype=dtype,
    )
    
    cdf = lambda args: tf.reduce_mean(tf.map_fn(partial(single_sample_cdf, *args, verbose=verbose), samples))
    cdf = tf.map_fn(cdf, [mean, chol, lower, upper], fn_output_signature=dtype)
    
    return cdf

# Checking the Tallis formula

In [146]:
def tallis(
        mean: np.array,
        cov: np.array,
        b: np.array,
    ):
    
    assert mean.shape == b.shape
    
    Q, = mean.shape
    dtype = mean.dtype
    
    # Compute p
    p = mvn_cdf(
        mean=tf.zeros(shape=mean.shape, dtype=tf.float64)[None, ...],
        cov=tf.convert_to_tensor(cov, dtype=tf.float64)[None, ...],
        upper=tf.convert_to_tensor(b-mean, dtype=tf.float64)[None, ...],
    ).numpy()
    
    # Compute uvn and mvn
    uvn = np.zeros(shape=(Q,))
    mvn = np.zeros(shape=(Q,))
    
    for q in range(Q):
    
        # Compute c
        c = np.zeros(shape=(Q-1,))
        R = np.zeros(shape=(Q-1, Q-1))

        for u in range(Q):
            
            if u == q:
                continue

            else:
                l = u if u < q else u-1
                c[l] = (b[u] - mean[u]) - (b[q] - mean[q]) * cov[q, u] / cov[q, q]

                
        for u in range(Q):
            for v in range(Q):
            
                if u == q or v == q:
                    continue

                else:
                    l = u if u < q else u-1
                    k = v if v < q else v-1
                    R[l, k] = cov[u, v] - cov[q, u] * cov[q, v] / cov[q, q]
    
        c = tf.convert_to_tensor(c)
        R = tf.convert_to_tensor(R)
    
        uvn[q] = norm.pdf(b[q], loc=mean[q], scale=cov[q, q])
        
        print(f"{b[q]=}")
        print(f"{mean[q]=}")
        print(f"{cov[q, q]=}")
    
        mvn[q] = mvn_cdf(
            mean=tf.zeros_like(c)[None, ...],
            cov=R[None, ...],
            upper=c[None, ...],
        ).numpy()[0]
        
    exp = np.zeros(shape=(Q,))
    
    for q in range(Q):
        
        exp[q] = exp[q] + mean[q]
            
        for i in range(Q):
            exp[q] = exp[q] - p**-1 * cov[q, i] * uvn[q] * mvn[q]
        
    print(f"{p=}")
    print(f"{mean=}")
    print(f"{cov=}")
    print(f"{uvn=}")
    print(f"{mvn=}")
    
    return exp


def mc_tallis(
        mean: tf.Tensor,
        cov: tf.Tensor,
        b: tf.Tensor,
        num_samples: int,
    ):
    
    normal = tfd.MultivariateNormalFullCovariance(
        loc=mean,
        covariance_matrix=cov,
    )
    
    samples = normal.sample(sample_shape=[num_samples])
    b = tf.tile(b[None, :], (num_samples, 1))
    
    bools = tf.reduce_all(tf.math.less(samples, b), axis=1)
    
    return tf.reduce_mean(samples[bools], axis=0)

In [147]:
tf.random.set_seed(0)
dtype = tf.float64

Q = 2
num_samples = int(1e6)

mean = tf.zeros(shape=(Q,), dtype=dtype)
rand = tf.random.uniform((Q, Q), dtype=dtype)
cov = 0. * tf.matmul(rand, rand, transpose_b=True) + tf.eye(Q, dtype=dtype)

b = tf.zeros_like(mean)

mc_tallis(
    mean=mean,
    cov=cov,
    b=b,
    num_samples=num_samples,
)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.79868892, -0.79754841])>

In [148]:
tallis(
    mean=mean,
    cov=cov,
    b=b,
)

b[q]=<tf.Tensor: shape=(), dtype=float64, numpy=0.0>
mean[q]=<tf.Tensor: shape=(), dtype=float64, numpy=0.0>
cov[q, q]=<tf.Tensor: shape=(), dtype=float64, numpy=1.0>
b[q]=<tf.Tensor: shape=(), dtype=float64, numpy=0.0>
mean[q]=<tf.Tensor: shape=(), dtype=float64, numpy=0.0>
cov[q, q]=<tf.Tensor: shape=(), dtype=float64, numpy=1.0>
p=array([0.25])
mean=<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 0.])>
cov=<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1., 0.],
       [0., 1.]])>
uvn=array([0.39894228, 0.39894228])
mvn=array([0.5, 0.5])


array([-0.79788456, -0.79788456])

In [149]:
samples = np.random.normal(0, 1, 100000)
np.mean(samples[np.where(samples < 0.)])

-0.7947068829535809